In [29]:
import math
import uni_mathfunc as unim

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'


<img src="img/pricetick1.png" width="400" height="340"> | <img src="img/pricetick2.png" width="400" height="340">


In [2]:
print(f"current price: {unim.price(1, 5000)}")
print(f"lower price: {unim.price(1, 4545)}")
print(f"upper price: {unim.price(1, 5500)}")

current price: 70.71067811865476
lower price: 67.4166151627327
upper price: 74.16198487095663


In [3]:
print(f"current tick: {unim.price_to_tick(unim.price(1,5000))}")
print(f"lower tick: {unim.price_to_tick(unim.price(1,4545))}")
print(f"upper tick: {unim.price_to_tick(unim.price(1,5500))}")

current tick: 85176
lower tick: 84222
upper tick: 86129


In [4]:
print(
    f"price USDC/ETH::5000/1 stored on smart contract: {unim.price_on_contract(unim.price(1, 5000))}"
)
print(
    f"price USDC/ETH::4545/1 stored on smart contract: {unim.price_on_contract(unim.price(1, 4545))}"
)
print(
    f"price USDC/ETH::5500/1 stored on smart contract: {unim.price_on_contract(unim.price(1, 5500))}"
)

price USDC/ETH::5000/1 stored on smart contract: 5.602277097478614e+30
price USDC/ETH::4545/1 stored on smart contract: 5.341294542274603e+30
price USDC/ETH::5500/1 stored on smart contract: 5.875717789736565e+30


概念：<span style="color:cyan">藍色線段</span> 注入流動性，C 點會往上推，代表 <span style="color:cyan">∆Y</span> 增加，所以 <span style="color:cyan">Y 幣</span> 增加流動性

概念：<span style="color:#8FBC8F">綠色線段</span> 注入流動性，C 點會往下推，代表 <span style="color:#8FBC8F">∆X</span> 增加，所以 <span style="color:#8FBC8F">X 幣</span> 增加流動性


<img src="img/liquidity.png" width="400" height="340"> | <img src="img/liquidity2.png" width="400" height="340">


## 個人解讀：

a 點 --> X 幣的最低價 (Y 幣最高價) ||| b 點 --> X 幣的最高價（Y 幣最低價）

current price 往右推 --> 池子只需提供 Y 幣 ||| current price 往左推 --> 池子只需提供 X 幣

當提供流動性時，我們希望價格穩定在 current price，所以要計算在藍綠兩個線段的流動性，並選擇較小當作 target liquidity

再根據 target liquidity 去計算要兩個線段各要注入多少 token (單一線段注入流動性只需一種 token)

<img src="img/calliquid1.png" width="400" height="350"> | <img src="img/calliquid3.png" width="400" height="350">


In [5]:
ether = 10**18

In [6]:
liq_x = unim.calc_liq_x(1 * ether, unim.price(1, 5500), unim.price(1,5000))
liq_x

1519437308014769733632

In [7]:
liq_y = unim.calc_liq_y(5000 * ether, unim.price(1, 4545), unim.price(1,5000))
liq_y

1517882343751509868544

In [8]:
# in fact we use the minimum liquidity when adding liuquidity to pool
min(liq_y, liq_x)

1517882343751509868544

<img src="img/liquidity2tokenAmount.png" width="400" height="350">

In [9]:
target_liquidity = min(liq_y, liq_x)
target_liquidity

1517882343751509868544

In [12]:
# 理解：在給定的 liquidity 下，若要把價格從 5500 推到 5000, 所需要 token X 數量
# 把價格從與 Y 軸交集的點推到曲線上，增加了 ∆X
ETH_in_amount = unim.calc_deposit_token_x(target_liquidity, unim.price(1, 5000), unim.price(1, 5500))
ETH_in_amount

998976618347425408

In [13]:
# 理解：在給定的 liquidity 下，若要把價格從 4545 推到 5000, 所需要 token Y 數量
# 把價格從與 X 軸交集的點推到曲線上，增加了 ∆Y
USDC_in_amount = unim.calc_deposit_token_y(target_liquidity, unim.price(1, 5000), unim.price(1, 4545))
USDC_in_amount

5000000000000000000000

## Swapping


In [15]:
swap_in_USDC = 42 * ether

In [39]:
# delta_price * liquidity = delta_token_y
delta_p_USDC = swap_in_USDC / target_liquidity
new_p_USDC = (unim.price(1, 5000) + delta_p_USDC)
new_tick = unim.price_to_tick(new_p_USDC)
new_p_USDC_X96 = int(unim.price_on_contract(new_p_USDC))

print(f"∆USDC_PRICE: {delta_p_USDC}")
print(f"NEW_USDC_PRICE: {new_p_USDC}")
print(f"NEW_USDC_PRICE_CORRSPONDING_TICK: {new_tick}")
print(f"NEW_USDC_PRICE_X96: {new_p_USDC_X96}")

# 在此例中，當前價格 5000 的 liquidty 就是我們前面算出來的 target_liquidity
# 若要把當前價格 5000 推到 new_p_USDC 的價格，所需的 token 量
# 要造成此例價格變動有兩種方式(?：
# 1. 丟 USDC 進池子, 換出 ETH 
# // 2. 從池子取出 ETH
USDC_in_amount = unim.calc_deposit_token_y(target_liquidity, unim.price(1,5000), new_p_USDC)
print(f"USDC_in_amount: {USDC_in_amount/ether} ether")
ETH_out_amount = unim.calc_deposit_token_x(target_liquidity, unim.price(1,5000), new_p_USDC) 
print(f"ETH_out_amount: {ETH_out_amount/ether} ether")

∆USDC_PRICE: 0.02767012882974529
NEW_USDC_PRICE: 70.7383482474845
NEW_USDC_PRICE_CORRSPONDING_TICK: 85184
NEW_USDC_PRICE_X96: 5604469350942327493700597317632
USDC_in_amount: 41.99999999999241 ether
ETH_out_amount: 0.008396714242160928 ether


<img src="img/verify_eth_out_amount.png">

In [59]:
# 透過上面的公式驗算 eth out amount
ETH_out_amount_verfiy = (1/new_p_USDC - 1/unim.price(1,5000)) * target_liquidity 
print(f"ETH_out_amount_verfiy: {ETH_out_amount_verfiy/ether} ether")

ETH_out_amount_verfiy: -0.008396714242162704 ether
